In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 9

In [4]:
name = list[k].getName()
name

'RP38'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.8f,%.8f] " % ( resultsMC[4], resultsMC[5])


b = "[%.8f,%.8f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.8f,%.8f] " % ( resultSubset[4], resultSubset[5])


In [8]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.00649444,0.00965021] ', None],
 [None, None, None, '[0.00649444,0.00965021] ', None],
 [None, None, None, '[0.00649444,0.00965021] ', None],
 [None, None, None, '[0.00649444,0.00965021] ', None],
 [None, None, None, '[0.00649444,0.00965021] ', None],
 [None, None, None, '[0.00649444,0.00965021] ', None]]

In [9]:
l = np.array([[1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1],[1, 1, 1, a, 1, 1]])

In [10]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité exacte de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  result[i][2]

results[0][3] = "-----------------"
results[1][3] = "-----------------"
results[2][3] = a

results[3][3] = c
 
results[4][3] = b


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2,x3,x4,x5,x6,x7]->[15.59 * 1e4 - x1 *x2^3 / (2 * x3^3) *((x4^2 - 4 * x5 * x6 * x7^2 + x4 * (x6 + 4 * x5 + 2 *x6 * x7)) / (x4 * x5 * (x4 + x6 + 2 *x6 *x7)))]


,Probabilité exacte de RP38,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,8.081249e-03,8.072328e-03,8.921258e-06,-----------------,18582,2.957052403913674
FORM,8.081249e-03,2.7934691e-02,1.9853441e-02,-----------------,101,-0.39035729345546955
SORM,8.081249e-03,2.824548e-02,2.0164233e-02,"[0.00649444,0.00965021]",101,-0.39710318072398376
FORM-IS,8.081249e-03,9.01761e-03,9.363602e-04,"[0.00719760,0.00902216]",469,0.9360355557216756
SUBSET,8.081249e-03,8.109882e-03,2.8632683e-05,"[0.00725692,0.01077829]",30000,2.4506164674480475


In [11]:
df.to_csv('problem38.csv')